In [1]:

import os
import geopandas as gpd
from geopandas import GeoDataFrame

project_crs = 'epsg:3857'

from shapely.geometry import LineString

import warnings

warnings.filterwarnings(action='ignore')

pjr_loc = os.path.dirname(os.getcwd())
from Code.general_functions import Preprocessing
import pandas as pd

In [34]:
# Join status field into the shapefile
places = [
    ('Turin_Italy','Turin'),
    # ('Tel_Aviv','TA'),
    #('San_Francisco__California','SF')
]

# Loop through each place to process the data
for place in places:
    # Load the selected streets from a CSV file
    selected_streets_csv =pd.read_csv(os.path.join(pjr_loc, f'places/{place[0]}/selected_streets_{place[1]}.csv')).set_index('Street name')
    selected_streets = gpd.read_file(os.path.join(pjr_loc, f'places/{place[0]}/selected_streets.shp')).set_index('name')
    selected_streets['Status'] = selected_streets_csv['Status']
    selected_streets.to_file(os.path.join(pjr_loc, f'places/{place[0]}/selected_streets.shp'))

    
selected_streets


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\18059\\OneDrive - ariel.ac.il\\Current_research\\SOD\\Code\\places/Turin_Italy/selected_streets_Turin.csv'

In [ ]:
# The next cell reads SOD network and select only these in the csv file

# Define the locations and their corresponding directory names
places = [
    ('Turin_Italy', 'Turin_Italy'),
    ('San_Francisco__California', 'San_Francisco__California'),
    ('Tel_Aviv', 'Tel_Aviv')
]
    #('Tel_Aviv', 'Tel_Aviv')
    # ('Turin__Italy', 'Turin'),
    # ('San_Francisco__California', 'San_Francisco'),
# Loop through each place to process the data
for place in places:
    # Load the selected streets from a CSV file
    selected_streets = pd.read_csv(os.path.join(pjr_loc, f'places/more/{place[0]}/selected_streets.csv'))
    
    # Load the street network shapefile as a GeoDataFrame
    sod_data = gpd.read_file(os.path.join(pjr_loc, f'places/{place[1]}/network_new.shp'))
    
    # Filter the GeoDataFrame to include only the streets in the selected streets list
    filtered_streets = sod_data[sod_data['name'].isin(selected_streets['Street name'])]
    
    # Dissolve the filtered streets by the 'name' column to aggregate geometries
    dissolved_streets = filtered_streets.dissolve(by='name')
    
    # Select only the 'geometry' column to save to the GeoPackage
    dissolved_streets = dissolved_streets[['geometry']]
    
    # Save the dissolved streets to a GeoPackage file
    output_path = os.path.join(pjr_loc, f'places/{place[1]}/selected_streets.shp')
    dissolved_streets.to_file(output_path)
    print(f"Processed and saved selected streets for {place[1]} to {output_path}")



In [ ]:
### This code find the number of polylines being aggregated and the results of the aggregation with the number of totals streets being aggregated
# Define the working folder path
work_folder = f'{pjr_loc}/places/Turin_Italy'

# Read the simplified shapefile into a GeoDataFrame
simp = gpd.read_file(f'{work_folder}/simp.shp')

# Filter the GeoDataFrame to include only simplified features
is_simplified = simp[simp['is_simplif'] == 1]

# Group the simplified features by 'name' and count the number of occurrences for each name
agg_streets = is_simplified.groupby('name').size()

# Read the original shapefile before simplification into a GeoDataFrame
db_before = gpd.read_file(f'{work_folder}/before_df.shp')

# Filter the original GeoDataFrame to include only the features with names that were aggregated in the simplified GeoDataFrame
only_aggregated = db_before[db_before['name'].isin(agg_streets.reset_index()['name'])]

# Output the result (optional, for verification purposes)
print(only_aggregated)

In [ ]:
# This code counts the number of roundabouts being aggregated and processes their geometric centroids.

# Define the working folder path
work_folder = f'{pjr_loc}/places/Turin_Italy'

# Initialize preprocessing with the specified place and test mode
place = 'Turin_Italy'
my_preprocessing = Preprocessing(place, is_test=True)

# Create a data folder for preprocessing
data_folder = my_preprocessing.create_folder()

# Perform the first filtering step
df_pro = my_preprocessing.first_filtering()

# Read the roundabout file and reproject to the project CRS
roundabout_file = my_preprocessing.round_about
roundabout_file = roundabout_file.to_crs(project_crs)

# Create a buffered geometry around each roundabout
round_about_buffer = roundabout_file['geometry'].buffer(cap_style=1, distance=10, join_style=1).unary_union

# Initialize a list to store the names and centroids of roundabouts
names = []
centroids = []

# Check if the buffer result is a single Polygon or multiple geometries
if round_about_buffer.type == 'Polygon':  # In case we have only one polygon
    names.append(0)
    centroids.append(round_about_buffer.centroid)
else:
    for idx, geom in enumerate(round_about_buffer.geoms):
        names.append(idx)
        centroids.append(geom.centroid)

# Create a GeoDataFrame from the lists with the specified CRS
centroid = GeoDataFrame({'name': names, 'geometry': centroids}, crs=project_crs)

# Output the resulting GeoDataFrame (optional, for verification purposes)
centroid

In [18]:
#('Tel_Aviv', 'Tel_Aviv')

# ('Turin_Italy', 'Turin'),
# ('San_Francisco__California', 'San_Francisco'),
## Choose locations and create folders if necessary
place  = 'Turin_Italy'
data_folder = os.path.join(pjr_loc, f'places/{place}')
os.makedirs(f'{data_folder}/csv/', exist_ok=True) # Here the csv files will be stored
before_df= gpd.read_file(f'{data_folder}/before_df.shp')
after_df = gpd.read_file(f'{data_folder}/network_new.shp').drop(columns='index')
# step 1-4 as one def
def street_its_connections(network,is_simplified=False):
    """
    Perform a spatial join to identify all streets connected to each street in the dataset.
    :param is_simplified:Decide whether to store the status of 'is_simplified'.
    :param network:
    :return:
    """
    # vars
    str_name = 'name_left'
    con_str_name = 'name_right'
    # 1.	Copy network
    df_analysis = network.copy()
    # 2.	Internal intersection
    s_join_analysis = gpd.sjoin(df_analysis,network)
    # 3.	Delete those with the same name

    s_join_analysis2 = s_join_analysis[s_join_analysis[str_name]!=s_join_analysis[con_str_name]]
    # 4.	For each group: name: list
    group_name = s_join_analysis2.groupby(str_name)
    dic_data={}
    def connected_streets(str_lines):
        """
        Populate the dictionary with the name of the current street and all its connected streets.
        :param str_lines:
        :return:
        """
        if is_simplified: # after SOD process we also want to know if all/part of the street was simplified.
            dic_data[str_lines[str_name].iloc[0]] =(list(str_lines[con_str_name].unique()),True if (str_lines['is_simplif_left'] == 1).any() else False)
        else:
            dic_data[str_lines[str_name].iloc[0]] =(list(str_lines[con_str_name].unique()),False)
    group_name.apply(connected_streets)
    return dic_data
# Create dictionary of street: connected streets before and after
dic_before = street_its_connections(before_df)
print('dic_before')
dic_after = street_its_connections(after_df,is_simplified=True)
print('dic_after')
# 5. Compare between the two dictionaries
df1 = pd.DataFrame(index= dic_after.keys(), data=dic_after.values(),columns=['ConnectedStreets','is_simplified']).reset_index(names='StreetName')
df2 = pd.DataFrame(index= dic_before.keys(), data=dic_before.values(),columns=['ConnectedStreets','is_simplified']).reset_index(names='StreetName')
def common_streets(row):
    """
    This function compares the connected streets of a given street in the current row with those in another DataFrame (df2).
    It returns the count of common connected streets.

    Parameters:
    row (pandas.Series): A row from the original DataFrame containing 'StreetName' and 'ConnectedStreets' columns.

    Returns:
    int: The number of common connected streets between the given row and the corresponding street in df2.
    """
    # Filter df2 to find the streets with the same name as the current row's street
    streets_in_old = df2[df2['StreetName'] == row['StreetName']]
    
    # Check if there are no matching streets in df2
    if streets_in_old.empty:
        return 0
    else:
        # Calculate the intersection of connected streets between the current row and the matched street in df2
        return len(set(row['ConnectedStreets']) & set(streets_in_old['ConnectedStreets'].iloc[0]))

def unique_streets_in_df1(row):
    """
    This function compares the connected streets of a given street in the current row with those in another DataFrame (df2).
    It returns the count of unique connected streets that are present in the current row but not in df2.

    Parameters:
    row (pandas.Series): A row from the original DataFrame containing 'StreetName' and 'ConnectedStreets' columns.

    Returns:
    int: The number of unique connected streets that are in the current row but not in the corresponding street in df2.
    """
    # Filter df2 to find the streets with the same name as the current row's street
    streets_in_old = df2[df2['StreetName'] == row['StreetName']]
    
    # Check if there are no matching streets in df2
    if streets_in_old.empty:
        return 0
    else:
        # Calculate the unique connected streets in the current row that are not in the matched street in df2
        return len(set(row['ConnectedStreets']) - set(streets_in_old['ConnectedStreets'].iloc[0]))


def unique_streets_in_df2(row):
    """
    This function compares the connected streets of a given street in the current row with those in another DataFrame (df2).
    It returns the count of unique connected streets that are present in df2 but not in the current row.

    Parameters:
    row (pandas.Series): A row from the original DataFrame containing 'StreetName' and 'ConnectedStreets' columns.

    Returns:
    int: The number of unique connected streets that are in the corresponding street in df2 but not in the current row.
    """
    # Filter df2 to find the streets with the same name as the current row's street
    streets_in_old = df2[df2['StreetName'] == row['StreetName']]
    
    # Check if there are no matching streets in df2
    if streets_in_old.empty:
        return 0
    else:
        # Calculate the unique connected streets in the matched street in df2 that are not in the current row
        return len(set(streets_in_old['ConnectedStreets'].iloc[0]) - set(row['ConnectedStreets']))


df1['CommonStreetsCount'] = df1.apply(common_streets, axis=1)
df1['UniqueStreetsInAfterCount'] = df1.apply(unique_streets_in_df1, axis=1)
df1['UniqueStreetsInBeforeCount'] = df1.apply(unique_streets_in_df2, axis=1)

df1['rate_success'] =(df1['CommonStreetsCount']/df1[['CommonStreetsCount','UniqueStreetsInAfterCount','UniqueStreetsInBeforeCount']].sum(axis=1)*100).round(0)
    
# Calculate rate success
df1['rate_success'] =(df1['CommonStreetsCount']/df1[['CommonStreetsCount','UniqueStreetsInAfterCount','UniqueStreetsInBeforeCount']].sum(axis=1)*100).round(0)

# Update the original data source
after_df2 = after_df.set_index('name')
after_df2['rate_success'] = df1.set_index('StreetName')['rate_success']
after_df2 = after_df2.reset_index().dissolve(by='name')
after_df2.to_file(f'{data_folder}/after_df.shp')
df1.to_csv(f'{data_folder}/csv/after_df.csv')
df2.to_csv(f'{data_folder}/csv/before_df.csv')
rate_success= 'rate_success'
is_simplified= 'is_simplified'
print(f'mean: {df1[rate_success].mean()}')
print(f'mean_simplified: {df1[df1[is_simplified]][rate_success].mean()}')
print(f'mean_no_simplified: {df1[~df1[is_simplified]][rate_success].mean()}')

dic_before
dic_after
mean: 80.42657657657658
mean_simplified: 75.36992840095465
mean_no_simplified: 81.60299833425874


In [16]:
place  = 'Turin_Italy'
data_folder = os.path.join(pjr_loc, f'places/{place}')
after_df2 = gpd.read_file(f'{data_folder}/after_df.shp')
len(after_df2[after_df2['rate_succe']== 100] )/len(after_df2)*100
# len(after_df2[(after_df2['is_simplif'] == 1) & (after_df2['rate_succe']== 100)] )/len(after_df2)*100

38.655462184873954

0       None
1       None
2       None
3       None
4       None
        ... 
2256    None
2257    None
2258    None
2259    None
2260    None
Length: 1940, dtype: geometry

In [ ]:
df1[df1]

In [ ]:
after_df2[after_df2]

In [ ]:
df = df1.copy()

# Define the bins and labels for the groups
bins = [0, 25, 50, 75, 100]
labels = ['0-25', '25-50', '50-75', '75-100']

# Create a new column with the group intervals
df['group'] = pd.cut(df['rate_success'], bins=bins, labels=labels, right=True)

# Calculate the percentage of entries in each group for All
group_percentages = df['group'].value_counts(normalize=True) * 100
group_percentages.name = 'Percentage'

# Split the DataFrame based on the boolean_value
df_true = df[df['is_simplified'] == True]
df_false = df[df['is_simplified'] == False]

# Calculate the percentage of entries in each group for True
group_percentages_true = df_true['group'].value_counts(normalize=True) * 100
group_percentages_true.name = 'Percentage_True'

# Calculate the percentage of entries in each group for False
group_percentages_false = df_false['group'].value_counts(normalize=True) * 100
group_percentages_false.name = 'Percentage_False'

# Combine the results into one DataFrame
output_df = pd.concat([group_percentages_true, group_percentages_false,group_percentages], axis=1).reset_index()
output_df.columns = ['Group','Percentage', 'Percentage_True', 'Percentage_False']

# Save the result to a CSV file
output_df.to_csv(f'{data_folder}/csv/group_percentages_by_boolean.csv', index=False)

print(output_df)


In [37]:
output_df.to_csv(f'{data_folder}/csv/group_percentages_by_boolean.csv', index=False)


In [ ]:
# Create dictionaries - by steps
# params
is_simplified = True
# vars
str_name = 'name_left'
con_str_name = 'name_right'
# 1.	Copy network
df_analysis = before_df.copy()
# 2.	Internal intersection
s_join_analysis = gpd.sjoin(df_analysis,before_df)

# 3.	Delete those with the same name
s_join_analysis2 = s_join_analysis[s_join_analysis[str_name]!=s_join_analysis[con_str_name]]

# 4.	For each group: name: list
group_name = s_join_analysis2.groupby(str_name)
dic_data={}
def connected_streets(str_lines):
    """
    Populate the dictionary with the name of the current street and all its connected streets.
    :param str_lines:
    :return:
    """
    if is_simplified: # after SOD process we also want to know if all/part of the street was simplified.
        dic_data[str_lines[str_name].iloc[0]] =(list(str_lines[con_str_name].unique()),True if (str_lines['is_simplif_left'] == 1).any() else False)
    else:
        dic_data[str_lines[str_name].iloc[0]] =(list(str_lines[con_str_name].unique()),False)
group_name.apply(connected_streets)
dic_data

In [ ]:
# # 4.	For each group: name: list - Code for one group
dic_data={}
# loop on one group
str_lines = group_name.get_group('Corso Chieri')
dic_data[str_lines[str_name].iloc[0]] =list(str_lines[con_str_name].unique())
# Calculate  if it is simplified
gg = True if (str_lines['is_simplif_left'] == 1).any() else False


In [ ]:
# from csv to shpfile
my_gdf = gpd.read_file('places\San_Francisco__California\Streets___Active_and_Retired_20231024.csv')
# This line build a correct format of the geometry based on the data from the line column
my_gdf['geometry'] = my_gdf['line'].apply(lambda x:LineString([tuple(map(float, coords.split())) for coords in  re.split(r'\(|\)', x)[1].split(',')]))
my_gdf.crs = 'EPSG:4326'
my_gdf.to_file(r'places\San_Francisco__California\ref_shp')
my_gdf.explore()

<span style="color:red; font-size:20px"> Get 10% random sample of unique names</span>


In [ ]:
# This code selects 10% of the streets randomly  to analysis manually  
# parameters
first_time = False
for place in ['Tel_Aviv','San_Francisco__California','Turin_Italy']:
    # if place!='San_Francisco__California':
    #     continue
    name ='name'

    if first_time:
        # Get Dataframe and choose only the simplified ones
        df = gpd.read_file(f'places/{place}/simp.shp')


        # Select 20% randomly
        unique_names = df[df['is_simplif']==1]['name'].unique()
        selected_names = pd.DataFrame(pd.Series(unique_names).sample(frac=0.1, random_state=42))
        selected_names['is_same']=''
        selected_names.set_index(0, inplace=True,)
        selected_names.index.name= 'street'
    else:
        selected_names = pd.read_csv(f'places/{place}/selected_streets.csv').set_index('Street name')[['Status 1','Status 2']]

    # Save the selected original streets in new shp files (regular and dissolve)
    df_2 = gpd.read_file(f'places/{place}/osm_data.gpkg',layer = 'edges')
    if place=='Tel_Aviv':
        df_2[name] = df_2['name:en']
        df_2.drop(columns='name:en',inplace=True)
    res = df_2[df_2[name].isin(selected_names.index)]
    res.to_file(f'places/{place}/selected_streets.shp')
    res.dissolve(by=name).sort_values(by=name).to_file(f'places/{place}/selected_diss.shp')
    # Create df to be filled in manually
    code_to_repeat = "selected_names.loc['',:]= []"

    # Loop to repeat the code
    for street in selected_names.index:
        print(f"selected_names.loc['{street}',:]= []")

In [ ]:
# It is not updated
data_dict= {0:'Success',
            1: 'A few minor flaws',
            2: 'Partially successful',
            3: "Failed",
            4:'The street of the ref network is presented partially/fully by more then one lane',
            5: 'It was not mapped into the ref network',
            6: 'The distant between the two parallel lanes are more 60 meters',
            7: 'Complex street configurations that can be interpreted as either a success or failure, depend on the need',
            8: "If a polyline representing a roundabout lacks the accompanying tag 'roundabout=True', the code will refrain from converting it into a center point representation.",
            9: "Plazas are not treated as roundabouts in the current version",
            10: 'The algorithm fails to connect nearby polylines to the roundabout due to its excessive size.',
            11: 'In certain segments, the algorithm failed to preserve the original shape.',
            12: 'Part of the street was not mapped into original OSM network'
            }
df_dict_data = pd.Series(data_dict)
df_dict_data.to_csv(f'places/status_info.csv')


In [ ]:
# data Query
# Arrange the tables to align with the corresponding shapefile.
for place in [('Turin_Italy','Turin'),('Tel_Aviv','TA'),('San_Francisco__California','SF')]:
    print(place)
    selected_streets =pd.read_csv(os.path.join(pjr_loc, f'places/{place[0]}/selected_streets_{place[1]}.csv'))
    print(selected_streets['Status'].value_counts())
    print(selected_streets['Status'].value_counts()/len(selected_streets)*100)


<span style="color:red; font-size:20px"> Update the new table - DONT USE</span>

In [ ]:
# Arrange the tables to align with the corresponding shapefile.
for place in ['Tel_Aviv','San_Francisco__California','Turin__Italy']:
    print(place)
    selected_streets = pd.read_csv(f'places/{place}/selected_streets.csv').sort_values(by='Street name').reset_index(drop=True).to_csv(f'places/{place}/selected_streets.csv')


In [ ]:
# Update the current key
new_list = [0, 4, None, 5, 6, 7, 2, 1, 8, 9, 3, 10, 11, 12]
my_dict = {i:key for i,key in enumerate(new_list)}
for place in ['Tel_Aviv','San_Francisco__California','Turin__Italy']:
    print(place)
    selected_streets = pd.read_csv(f'places/{place}/selected_streets.csv')
    # Split the column based on commas and expand into new columns
    selected_streets[['status1', 'status2']] = selected_streets['is_same'].str.split(',',n=1, expand=True)

    # Drop the original column if needed
    selected_streets = selected_streets.drop('is_same', axis=1)

    # Update the status with new values
    selected_streets['status1']  = selected_streets['status1'].apply(lambda x:my_dict[int(x)])
    selected_streets['status2']  = selected_streets['status2'].apply(lambda x:my_dict[int(x)] if x is not None else '' )
    selected_streets.to_csv(f'places/{place}/selected_streets.csv')
    print(selected_streets)